In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
SEED = 314159
TRAIN_TEST_SPLIT = 0.80

data_path = "D:\data\machine_learning"

# Ансамбли: градиентный бустинг


In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(data_path+'/'+"winequality-red.csv")
df_major = df[df["quality"].isin([5,6])]
print("Length of filtered data is", len(df_major))
X = df_major.drop('quality', axis=1)
y = df_major['quality']
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)

# Градиентный бустинг

Градиентный бустинг - довольно мощная метамодель, с огромным количеством параметров и хитростей. Мы сегодня остановимся только на оссновных. Для начала рассмотрим самый стандартный бустинг с использованием деревьев решений (CART). Параметры базовых моделей такие же, как и раньше, но настройка амого бустинга довольно сложна!

Важный вопрос при обучении модели - какую функцию ошибок выбрать? Какая задача возникает при обработке датасета с вином?

Для того, чтобы оценивать модель, полезны различные метрики - численные характеристики ее качества. При этом бустинги настолько галантны, что предоставляют нам возможность оценивать метрики прямо при обучении. Для этого необходимо задать тип метрики в конструкторе и eval_set при запуске fit().

In [ ]:
model = XGBClassifier(
    objective="binary:logistic", n_estimators=100, learning_rate=1, seed=SEED,
    eval_metric="auc"
)
fit_params = {"eval_set":[(X_train, y_train),(X_test, y_test)], "verbose": False}
# Add verbose=False to avoid printing out updates with each cycle
model.fit(X_train, y_train,
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=False)

In [ ]:
results = model.evals_result()

In [ ]:
error_function = "auc"
plt.figure(figsize=(10,7))
plt.plot(results["validation_0"][error_function], label="Training loss")
plt.plot(results["validation_1"][error_function], label="Validation loss")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

Как мы видим, хотя лосс при обучении падал и падал, на валидации метрики перестали улучшаться довольно рано. Это довольно плохой знак. Однако говорит ли это о катастрофической ситуации? Проверим переобучение с помощью кросс-валидации.

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, y, cv=10, scoring=["accuracy"],
                            return_train_score=True)
print("Train F1 is", cv_results['train_accuracy'].mean())
print("Test F1 is", cv_results['test_accuracy'].mean())


Кажется, у нас действительно серьезные проблемы. Попробуем уменьшить скорость обучения.


In [ ]:
# train and eval model with smaller lr


In [ ]:
# plot results

Помогло ли это? Попробуем получить результаты лучше, поиграв с параметрами.

In [ ]:
model = XGBClassifier(
    # your params

)

# train, test and plot results

In [ ]:
# your code

Так как параметров довольно много, может быть разумно автоматизировать их поиск.

In [ ]:
from sklearn.model_selection import GridSearchCV

xgboost_params = {
    # set your params range
                 }
xgboost_best_grid = GridSearchCV(model, xgboost_params,
                                 cv=7, n_jobs=-1,
                                 return_train_score=True).fit(X_train, y_train,**fit_params)

In [ ]:
print(xgboost_best_grid.best_params_)

Давайте проверим, какую точность мы получим с лучшими параметрами.

In [ ]:
# train and test model

# Границы разбиения

Мы можем также , как и раньше, построить границы принятия решений для нашего бустинга. При этом мы даже можем использовать хорошо знакомый нам способ из sklearn. Давайте посмотрим, как будут меняться предсказания для разных параметров.

In [ ]:
# add plot_boundary method. Do we need to change it?

In [ ]:
from xgboost import XGBModel
from sklearn.decomposition import PCA
pca = # your code
# make pca decomposition of data
features = ["x1", "x2"]
model = # your model
# Add silent=True to avoid printing out updates with each cycle
fit_params = # set fit params
model.fit(X_train_pca, y_train,
            **fit_params)
plot_boundary(model, data=X_train_pca, features=features, y=y_train)

# Регрессия

Конечно, градиентный бустинг удобен не только для классификации, но и регрессии. На ее примере давайте и посмотрим, как бустинг формирует предсказания.
Построим простейшую зависимость.

In [ ]:
import seaborn as sns
data = pd.DataFrame()
data["x"] = np.linspace(-15, 15, 300)
data["y"] = # create your simple sinusoidal-like function
sns.scatterplot(x=data["x"], y=data["y"])

In [ ]:
from xgboost import XGBRegressor
n_estimators = 4
model = XGBRegressor(n_estimators=n_estimators)
# Add silent=True to avoid printing out updates with each cycle
model.fit(data["x"], data["y"], verbose=False)


Бустинг состоит из нескольких деревьев. Вопрос: какая операция над индивидуальными предсказаниями нужна, чтобы получить финальный ответ?

Проиллюстрируем это.

In [ ]:
import xgboost
from scipy.special import logit as inverse_sigmoid

booster_ = model.get_booster()

# Extract indivudual predictions. You will need to use xgboost.DMatrix(input_data)
individual_preds = []
for tree_ in booster_:
    # your code

Для начала выведем индивидуальные предсказания деревьев на график

In [ ]:
fig, ax = plt.subplots(len(individual_preds), 1)
for i, preds in enumerate(individual_preds):
    sns.scatterplot(x=data["x"], y=data["y"], ax=ax[i])
    sns.lineplot(x=data["x"], y=preds, ax=ax[i], c='g')

Теперь можно получить последовательность предсказаний уже для полной модели

In [ ]:
individual_preds = # your code

fig, ax = plt.subplots(len(individual_preds), 1)
for i, preds in enumerate(individual_preds):
    sns.scatterplot(x=data["x"], y=data["y"], ax=ax[i])
    # your code

In [ ]:
# Дополнительное задание: Построить индивидуальные предсказания и для классификации. Обратите внимание, что предсказания суммируются до взятия сигмоды. Необходимо сделать следущее:

# individual_logits = inverse_sigmoid(individual_preds)
# final_logits = indivudual_logits.sum(axis=0)


## Monotonic
В реальных задачах часто бывает так, что функциональная форма приемлемой модели каким-то образом ограничена. Так, часто модель ограничена так, чтобы сохранять монотонность. Вопрос: как математически описвается монотонность?

In [ ]:
data = pd.DataFrame()
data["x"] = np.linspace(-15, 15, 300)
data["y"] = # make your trendy data
sns.scatterplot(x=data["x"], y=data["y"])

In [ ]:
from xgboost import XGBRegressor
n_estimators = 10
model = XGBRegressor(n_estimators=n_estimators)
# fit-predict for your data.

В XGBoost очень просто обеспечить соблюдение монотонности. Для этого достаточно передать модели словарь с названиями фичей и соответствующим ограничением (+1 - возрастание, -1 - убывание).


In [ ]:
from xgboost import XGBRegressor
n_estimators = 10
model = XGBRegressor(n_estimators=n_estimators)  #add your constraints!
# fit-predict for your data.

Вопрос: что происходит с деревьями, когда на модель задаются ограничения?


In [ ]:
# Дополнительная задача - построить деревья до/после задания ограничения

## MSE vs MAE


In [ ]:
n_estimators = 10
# make two models
# Add silent=True to avoid printing out updates with each cycle
model_mae.fit(data["x"], data["y"], verbose=False)
model_mse.fit(data["x"], data["y"], verbose=False)
fig = plt.figure()
sns.scatterplot(x=data["x"], y=data["y"])
sns.lineplot(x=data["x"], y=model_mae.predict(data["x"]), c='g')
sns.lineplot(x=data["x"], y=model_mse.predict(data["x"]), c='r')

Какие выводы можно сделать о результатах предсказания в зависимости от выбора функции потерь? Что еще нужно посмотреть, чтобы сделать более уверенный вывод?

In [ ]:
# your code (optional)

## Real world example


Рассмотрим уже известный нам набор данных о стоимости жилья. Это также задача регрессии, причем тут представлены признаки не только непрерывные, но и дискретные.

In [ ]:
data = pd.read_csv(f"{data_path}/realestate.txt", sep="\t")
X = data.drop("SalePrice", axis=1)
y = data[["SalePrice"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)


In [ ]:
X_train.describe()

Посмотрим, как ведет себя модель c разными настройками

In [ ]:
n_estimators = 10
# train-test regressor

Мы можем явно задать категориальные фичи. Поменяется ли результат? Как думаете, почему?

In [ ]:
categorical_features=[] # select categorical_features
X.loc[:, categorical_features] = X[categorical_features].astype("category")

In [ ]:
n_estimators = 10
# train test regressor

## Interaction constraints
Когда глубина дерева больше единицы, многие переменные взаимодействуют исключительно ради минимизации потерь при обучении, и результирующее дерево решений может отражать ложные отношения (шум), а не реальные отношения, которые обобщаются для разных наборов данных. Ограничения взаимодействия признаков позволяют пользователям решать, каким переменным разрешено взаимодействовать, а каким нет.

Вопрос: для чего это может быть полезно?


In [ ]:
constraints = [
    #select constraints lists
]
# model
# Add silent=True to avoid printing out updates with each cycle
model.fit(X_train, y_train, verbose=False)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xgboost.plot_tree(model, num_trees=4, ax=ax)
plt.show()

Так, пользователи могут иметь предварительные знания о связях между различными признаками и кодировать их как ограничения во время построения модели. Но есть и некоторые тонкости в определении ограничений.

В качестве примера возьмем  набор ограничений [[1, 2], [2, 3, 4]]. Как мы видим, 2 признак появляется в обоих наборах: [1, 2] и [2, 3, 4]. Таким образом, фактический набор признаков, которым разрешено взаимодействовать с 2, равен [1, 3, 4].

Пусть корень дерева разделяется именно по 2. Поскольку все его потомки должны иметь возможность взаимодействовать с этим узлом, все 4 признака являются законными кандидатами на разделение на втором уровне. Вопрос: является ли это игнорированием исходных ограничений?

В качестве еще одного примера возьмем [[0, 1], [0, 1, 2], [1, 2]]. Давайте рассмотрим все возможные наборы для построения второго уровня.

Последний пример следующий: [[0, 1], [0, 2, 3]]. Возьмем 1 в качестве признака на 0 уровне. Какие возможные кандидаты могут быть на 1 уровне? А на 2?


## Промежуток предсказания
Вопрос: какие значения будет предсказывать бустинг в случае регрессии? Как вы думаете, что он предскажет для "трендовых" данных за пределами обучающего набора? А случайный лес?

А что будет предсказываться в случае классификации?

_Note. Иллюстрация есть в блоге https://medium.com/gousto-engineering-techbrunch/the-problem-with-gradient-boosting-gradient-boosted-gremlins-a69908dcea94_

## Случайный лес
Интересная особенность бустинга в том, что его можно настроить как случайный лес (при определенных усилиях). Более того, мы можем прямо в интерфейсе XGBoost построить комбинацию бустинга и случайного леса.

In [ ]:
n_estimators = 10
model = XGBRegressor(
    # make ranfom forest from GBoosting
)
# Add silent=True to avoid printing out updates with each cycle
model.fit(X_train, y_train, verbose=False)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xgboost.plot_tree(model, num_trees=4, ax=ax)
plt.show()

In [ ]:
n_estimators = 10
model = XGBRFRegressor() # your params
model.fit(X_train, y_train, verbose=False)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xgboost.plot_tree(model, num_trees=4, ax=ax)
plt.show()

Сравнимся со случайным лесом из Scikit-learn.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# train test sklearn random forest